In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from time import time

In [2]:
df1 = pd.read_csv('../data/match_data.csv')
X1 = df1.drop(columns=['Date', 'Team1', 'Team2', 'Score'])
y1 = df1['Score']
ds1_name = "match_data"

df2 = pd.read_csv('../data/cleaned_rounds_data.csv')
df2 = df2.drop(columns=[f'player_{i}_{suffix}' for i in range(1, 11) for suffix in ['team_name', 'name']])
X2 = df2.drop(['round_winner'], axis=1)
y2 = df2['round_winner']
ds2_name = "rounds_data"

df3 = pd.read_csv('../data/cleaned_rounds_data_with_stats.csv')
df3 = df3.drop(columns=[f'player_{i}_{suffix}' for i in range(1, 11) for suffix in ['team_name', 'name']])
X3 = df3.drop(columns=['round_winner'])
y3 = df3['round_winner']
ds3_name = "rounds_data_with_stats"

datasets = [(X1, y1, ds1_name), (X2, y2, ds2_name), (X3, y3, ds3_name)]

In [3]:
max_depth = [3, 5, 7, 9, 11]
learning_rate = [0.01, 0.03, 0.05, 0.1, 0.3]
gamma = [0, 0.1, 0.2, 0.3, 0.4]
reg_lambda = [0, 0.1, 0.2, 0.3, 0.4]

In [5]:
best_params = []
for X, y, ds_name in datasets:
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42, stratify=y_test)
    best_acc = 0
    best_md = 0
    best_lr = 0
    best_g = 0
    best_rl = 0
    time_start = time()
    for md in max_depth:
        for lr in learning_rate:
            for g in gamma:
                for rl in reg_lambda:
                    xgb = XGBClassifier(max_depth=md, learning_rate=lr, gamma=g, reg_lambda=rl)
                    xgb.fit(X_train, y_train)
                    acc = xgb.score(X_val, y_val)
                    if acc > best_acc:
                        print(f"{ds_name}: new best accuracy={acc} with max_depth={md}, learning_rate={lr}, gamma={g}, reg_lambda={rl}")
                        best_md = md
                        best_lr = lr
                        best_g = g
                        best_rl = rl
                        best_acc = acc
    xgb = XGBClassifier(max_depth=best_md, learning_rate=best_lr, gamma=best_g, reg_lambda=best_rl)
    xgb.fit(X_train, y_train)
    acc = xgb.score(X_test, y_test)
    time_end = time()
    print(f"Dataset: {ds_name}, Accuracy:{acc}, Best max_depth:{best_md}, Best learning_rate:{best_lr}, Best gamma:{best_g}, Best reg_lambda:{best_rl}, Duration:{time_end-time_start}")

    best_params.append((ds_name, best_md, best_lr, best_g, best_rl, acc))

match_data: new best accuracy=0.598825831702544 with max_depth=3, learning_rate=0.01, gamma=0, reg_lambda=0
match_data: new best accuracy=0.6046966731898239 with max_depth=3, learning_rate=0.05, gamma=0, reg_lambda=0.2
match_data: new best accuracy=0.6066536203522505 with max_depth=3, learning_rate=0.1, gamma=0.2, reg_lambda=0.3
match_data: new best accuracy=0.6144814090019569 with max_depth=3, learning_rate=0.1, gamma=0.4, reg_lambda=0.4
match_data: new best accuracy=0.6223091976516634 with max_depth=3, learning_rate=0.3, gamma=0, reg_lambda=0.4
match_data: new best accuracy=0.6281800391389433 with max_depth=3, learning_rate=0.3, gamma=0.2, reg_lambda=0.4
Dataset: match_data, Accuracy:0.598825831702544, Best max_depth:3, Best learning_rate:0.3, Best gamma:0.2, Best reg_lambda:0.4, Duration:45.460251569747925
rounds_data: new best accuracy=0.7472198460222412 with max_depth=3, learning_rate=0.01, gamma=0, reg_lambda=0
rounds_data: new best accuracy=0.7476475620188195 with max_depth=3, l

IN THE LOGS ABOVE FOR THE BEST PARAMETERS I PRINTED THE BEST ACCURACY NOT THE TEST ACCURACY
BELOW IS THE TEST ACCURACY

In [15]:
for ds_name, best_md, best_lr, best_g, best_rl, best_spw, acc in best_params:
    print(f"Dataset: {ds_name}, Accuracy:{acc}, Best max_depth:{best_md}, Best learning_rate:{best_lr}, Best gamma:{best_g}, Best reg_lambda:{best_rl}, Best scale_pos_weight:{best_spw}")

Dataset: match_data, Accuracy:0.598825831702544, Best max_depth:3, Best learning_rate:0.3, Best gamma:0.2, Best reg_lambda:0.4, Best scale_pos_weight:1
Dataset: rounds_data, Accuracy:0.8258451005562687, Best max_depth:9, Best learning_rate:0.1, Best gamma:0.3, Best reg_lambda:0.3, Best scale_pos_weight:1
Dataset: rounds_data_with_stats, Accuracy:0.8275566966195977, Best max_depth:9, Best learning_rate:0.05, Best gamma:0.3, Best reg_lambda:0, Best scale_pos_weight:1
